# Deploy Supply Chain Agent
This notebook demonstrates how to deploy the agent.

## Cluster Configuration
This notebook was tested on the following Databricks cluster configuration:
- **Databricks Runtime Version:** 16.4 LTS ML (includes Apache Spark 3.5.2, Scala 2.12)
- **Single Node** 
    - Azure: Standard_DS4_v2 (28 GB Memory, 8 Cores)
    - AWS: m5d.2xlarge (32 GB Memory, 8 Cores)

In [0]:
%pip install -r ../requirements.txt --quiet
dbutils.library.restartPython()

## Load the agent

In [0]:
from databricks import agents
from mlflow import MlflowClient
import mlflow

client = MlflowClient()
# Connect to the Unity catalog model registry
mlflow.set_registry_uri("databricks-uc")

catalog = "supply_chain_stress_test"    # Change here
schema = "agents"                       # Change here
agent_name = "supply_chain_agent"       # Change here
model_info = client.get_model_version_by_alias(f"{catalog}.{schema}.{agent_name}", "production")

user_email = spark.sql('select current_user() as user').collect()[0]['user']  # User email address
first_name = user_email.split('.')[0]                                         # User first name

mlflow.set_experiment(f"/Users/{user_email}/supply-chain-agent")

## Deploy the agent

In [0]:
import os
import mlflow
from dbruntime.databricks_repl_context import get_context

# TODO: set WORKSPACE_URL manually if it cannot be inferred from the current notebook
WORKSPACE_URL = None
if WORKSPACE_URL is None:
  workspace_url_hostname = get_context().browserHostName
  assert workspace_url_hostname is not None, "Unable to look up current workspace URL. This can happen if running against serverless compute. Manually set WORKSPACE_URL yourself above, or run this notebook against classic compute"
  WORKSPACE_URL = f"https://{workspace_url_hostname}"

# TODO: set secret_scope_name and secret_key_name to access your PAT
secret_scope = first_name
secret_key = "token"

In [0]:
# Deploy to enable the review app and create an API endpoint
deployment_info = agents.deploy(
    f"{catalog}.{schema}.{agent_name}", 
    model_info.version,
    environment_vars={
        "HOST": f"{WORKSPACE_URL}",
        "TOKEN": f"{{{{secrets/{secret_scope}/{secret_key}}}}}",
    },
)

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/deploy-agent) for details